In [0]:
from pyspark.sql.functions import col, sum, count, when, desc, rank
from pyspark.sql.window import Window


In [0]:
%run ../config/path_variables

In [0]:
race_results_df = spark.read.format("delta").load(f"{presentation_file_path}/race_results")

In [0]:
display(race_results_df)

In [0]:
constructor_standings_df = race_results_df.groupBy("race_year", "team") \
                                        .agg(sum("points").alias("total_points"),
                                             count(when(col("position") == 1, True)).alias("wins"))

In [0]:
constructor_rank_spec = Window.partitionBy("race_year").orderBy(desc("total_points"), desc("wins"))
final_df = constructor_standings_df.withColumn("rank", rank().over(constructor_rank_spec))

In [0]:
display(final_df.filter("race_year = 2020"))


In [0]:
final_df.write.mode("overwrite").format("delta").saveAsTable("f1_presentation.constructor_standings")